In [26]:
import advent

maps = advent.get_lines_doublenewline(5)


In [27]:
def is_in_range(range: str, num: int):
    # range looks like "x y z"
    _, source, length = range.split(" ")
    return num >= int(source) and num < (int(source) + int(length))

def apply_range(range: str, num: int):
    # ASSUMES ITS IN RANGE!
    dest, source, _ = range.split(" ")
    return num + int(dest) - int(source)

def apply_map(map: list[str], num: int):
    # assume map[0] is just a string, ignore it
    for range in map[1:]:
        if is_in_range(range, num): return apply_range(range, num)
    return num

print(apply_map(['seed-to-soil map:', '50 98 2', '52 50 48'], 99))
# should be 51

51


In [28]:
def apply_all_maps(data: list[list[str]], num: int):
    for map in data[1:]:
        num = apply_map(map, num)
    return num

nums = maps[0][0].split(": ")[1].split(" ")

results = [apply_all_maps(maps, int(num)) for num in nums]
print(min(results))

178159714
